In [1]:
# Term Deposit 
# Author: Avisheka Guha Niyogi

In [1]:
# Importing Libraries

import pandas as pd
import numpy as np
from random import randint
import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline
from scipy.stats import norm
from sklearn.preprocessing import StandardScaler
from sklearn.feature_selection import VarianceThreshold
from sklearn.model_selection import train_test_split
from sklearn.tree import DecisionTreeClassifier
from sklearn.model_selection import GridSearchCV
from sklearn.ensemble import AdaBoostClassifier, GradientBoostingClassifier, RandomForestClassifier
from xgboost import XGBClassifier
from sklearn.metrics import confusion_matrix
from sklearn.svm import SVC
from sklearn.metrics import accuracy_score
from sklearn.metrics import classification_report
from sklearn.metrics import roc_curve, auc
from sklearn.metrics import roc_auc_score, roc_curve
from sklearn.neighbors import KNeighborsClassifier 
from sklearn.linear_model import LogisticRegression
from sklearn.naive_bayes import BernoulliNB
from sklearn import metrics
from sklearn.metrics import precision_recall_fscore_support as score, precision_score, recall_score, f1_score
from scipy.stats import ttest_ind
from statsmodels.stats import weightstats as stests
import statsmodels.api as sm
from statsmodels.formula.api import ols
from statsmodels.stats.multicomp import pairwise_tukeyhsd
from statsmodels.stats.multicomp import MultiComparison
from sklearn.model_selection import RandomizedSearchCV
from scipy.stats import randint
import warnings
warnings.filterwarnings("ignore")

In [3]:
# Importing data

ds=pd.read_excel(r"C:\Users\hp\LearnBayAIML\PythonCodes\Projects\Time-Series\yen.xlsx")

In [4]:
# Trim the dataset to begin on January 1st, 1990
dsn = ds.loc["1984-01-01":, :]
tsn= ds.loc["1984-01-01":, :]
dsn.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 8917 entries, 1985 to 10901
Data columns (total 9 columns):
 #   Column                      Non-Null Count  Dtype         
---  ------                      --------------  -----         
 0   Date                        8917 non-null   datetime64[ns]
 1   Open                        8917 non-null   int64         
 2   High                        8917 non-null   int64         
 3   Low                         8917 non-null   int64         
 4   Last                        8917 non-null   int64         
 5   Change                      1415 non-null   float64       
 6   Settle                      8917 non-null   int64         
 7   Volume                      8917 non-null   int64         
 8   Previous Day Open Interest  8917 non-null   int64         
dtypes: datetime64[ns](1), float64(1), int64(7)
memory usage: 627.1 KB


In [27]:
dsn.isna().sum()

Date                             0
Open                             0
High                             0
Low                              0
Last                             0
Change                        7502
Settle                           0
Volume                           0
Previous Day Open Interest       0
dtype: int64

In [21]:
# Fill Nan values with median

dst['Change'].fillna(dst['Change'].median(),inplace=True)

In [28]:
dsn['Date'] = pd.to_datetime(dsn['Date'],format='%Y%m%d')

In [42]:
dsn['Year'] = pd.DatetimeIndex(dsn['Date']).year
dsn.loc[(dsn['Year'] >  1980) & (dsn['Year'] < 1991), 'YearBucket'] = 'B1'
dsn.loc[(dsn['Year'] >  1990) & (dsn['Year'] < 2001), 'YearBucket'] = 'B2'
dsn.loc[(dsn['Year'] >  2000) & (dsn['Year'] < 2011), 'YearBucket'] = 'B3'
dsn.loc[(dsn['Year'] >  2010) & (dsn['Year'] < 2021), 'YearBucket'] = 'B4'

dsn.groupby('YearBucket').agg({'Change': 'median'})

dsn['Change'].fillna(dsn['Change'].median(),inplace=True)

dsn

,Date,Open,High,Low,Last,Change,Settle,Volume,Previous Day Open Interest,Year,YearBucket
1985,1984-06-15,4298,4300,4286,4288,27.0,4288,3592,6529,1984,B1
1986,1984-06-18,4289,4289,4284,4285,27.0,4285,989,5983,1984,B1
1987,1984-06-19,4343,4354,4342,4350,27.0,4350,3278,15223,1984,B1
1988,1984-06-20,4344,4344,4315,4325,27.0,4325,6636,16238,1984,B1
1989,1984-06-21,4314,4325,4312,4318,27.0,4318,4144,15530,1984,B1
...,...,...,...,...,...,...,...,...,...,...,...
10897,2019-10-09,9381,9392,9330,9344,38.5,9338,99153,145470,2019,B4
10898,2019-10-10,9344,9380,9294,9301,34.5,9304,159397,144474,2019,B4
10899,2019-10-11,9308,9309,9240,9267,52.5,9251,158810,147471,2019,B4
10900,2019-10-14,9259,9292,9250,9261,14.0,9265,69457,153902,2019,B4


In [13]:
# ts=dsn.copy()
tsn['Change'].fillna(tsn['Change'].median(),inplace=True)

tsn['Return'] = (tsn[["Settle"]].pct_change() * 100)
# Create a lagged return using the shift function
tsn['Lagged_Return'] = tsn['Return'].shift()
tsn = tsn.dropna()
tsn.tail()

# ts.drop('YearBucket',axis=1)

tsn=tsn.replace(-np.inf, np.nan).dropna()

# Create a train/test split for the data using 2018-2019 for testing and the rest for training
train = tsn[:2017]
test = tsn[2018:]

tsn.tail()


,Date,Open,High,Low,Last,Change,Settle,Volume,Previous Day Open Interest,Return,Lagged_Return
10897,2019-10-09,9381,9392,9330,9344,38.5,9338,99153,145470,-0.405290,0.170940
10898,2019-10-10,9344,9380,9294,9301,34.5,9304,159397,144474,-0.364104,-0.405290
10899,2019-10-11,9308,9309,9240,9267,52.5,9251,158810,147471,-0.569647,-0.364104
10900,2019-10-14,9259,9292,9250,9261,14.0,9265,69457,153902,0.151335,-0.569647
10901,2019-10-15,9264,9280,9216,9220,43.5,9222,108342,151564,-0.464112,0.151335


In [14]:
X_train = train["Lagged_Return"].to_frame()
Y_train = train["Return"]
X_test = test["Lagged_Return"].to_frame()
Y_test = test["Return"]